<a href="https://colab.research.google.com/github/detektor777/colab_list/blob/main/embed_subtitles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Go to Google Drive and upload the video file to the root of the drive

https://drive.google.com/drive/


In [ ]:
#@title ##**Enter video file name** { display-mode: "form" }
inptut = input("Enter the file name and press enter (default: input.mp4) ") or "input.mp4"
video_filename = f"/content/drive/MyDrive/{inptut}"

In [ ]:
#@title ##**Connecting to Google Drive** { display-mode: "form" }
from google.colab import drive

drive.mount('/content/drive')
!cp "{output_filename}" "/content/drive/My Drive/"

In [ ]:
#@title ##**Install** { display-mode: "form" }
!apt-get install -y ffmpeg > /dev/null 2>&1

In [ ]:
#@title ##**Upload subtitles** { display-mode: "form" }
print("Please upload the subtitle file for the video")
uploaded_subtitles = files.upload()

subtitle_filename = next(iter(uploaded_subtitles))
print(f"Subtitle file '{subtitle_filename}' downloaded successfully")

In [ ]:
#@title ##**Run** { display-mode: "form" }

output_filename = "/content/drive/MyDrive/video_with_subtitles.mp4"

!ffmpeg -i "{video_filename}" -vf "subtitles={subtitle_filename}" "{output_filename}" -y

print(f"The processed video is saved as '{output_filename}'.")

In [ ]:
#@title ##**Download video** { display-mode: "form" }
from google.colab import files

files.download(output_filename)